In [144]:
# Montar uma função que construa o dataset desejado.

<h1 style='font-size:40px'> Stock Price Variation Predictor</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The present project aims to create a stock price variation predictor using the corporations' fundamentals. 
        </li>
        <li> 
            The model must estimate the given quarter's 80th percentile variation using the financial statements from the previous quarter.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Building the dataset</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Unlike most of the ML projects out there, in this one we'll face the challenge of making the dataset with which our models will carry out their predictions. Fortunately, the yfinance library can swiftly retrieve the corporate financial statements needed.
        </li>
    </ul>
</div>

In [142]:
from yfinance import Ticker
petr3 = Ticker('PETR3.SA')
pd.set_option('max_columns', None)

In [201]:
from yfinance import Ticker
petr3_quarterly_balance_sheet = petr3.quarterly_balance_sheet.T
petr3_quarterly_balance_sheet.index = pd.PeriodIndex(petr3_quarterly_balance_sheet.index, freq='Q')

petr3_quarterly_financials = petr3.quarterly_financials.T
petr3_quarterly_financials.index = pd.PeriodIndex(petr3_quarterly_financials.index, freq='Q')

petr3_quarterly_cashflow = petr3.quarterly_cashflow.T
petr3_quarterly_cashflow.index = pd.PeriodIndex(petr3_quarterly_cashflow.index, freq='Q')

petr3_quarterly_earnings = petr3.quarterly_earnings
petr3_quarterly_earnings.index = pd.PeriodIndex(petr3_quarterly_earnings.index, freq='Q')

In [193]:
dataset = (petr3_quarterly_cashflow
     .merge(petr3_quarterly_earnings, left_index=True, right_index=True)
     .merge(petr3_quarterly_financials, left_index=True, right_index=True)
     .merge(petr3_quarterly_balance_sheet, left_index=True, right_index=True)).sort_index()

In [192]:
dataset#.index[0].year

,Investments,Change To Liabilities,Total Cashflows From Investing Activities,Net Borrowings,Total Cash From Financing Activities,Change To Operating Activities,Net Income_x,Change In Cash,Effect Of Exchange Rate,Total Cash From Operating Activities,Depreciation,Other Cashflows From Investing Activities,Dividends Paid,Change To Inventory,Change To Account Receivables,Other Cashflows From Financing Activities,Change To Netincome,Capital Expenditures,Revenue,Earnings,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest_x,Net Income_y,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,Interest Expense,Extraordinary Items,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares,Intangible Assets,Total Liab,Total Stockholder Equity,Minority Interest_y,Other Current Liab,Total Assets,Common Stock,Other Current Assets,Retained Earnings,Other Liab,Good Will,Treasury Stock,Other Assets,Cash,Total Current Liabilities,Deferred Long Term Asset Charges,Short Long Term Debt,Other Stockholder Equity,Property Plant Equipment,Total Current Assets,Long Term Investments,Net Tangible Assets,Short Term Investments,Net Receivables,Long Term Debt,Inventory,Accounts Payable
2021Q2,3.540000e+08,-1.537000e+09,-5.187000e+09,-5.418100e+10,-6.552700e+10,-3.840000e+08,4.285500e+10,-1.902900e+10,-4.879000e+09,5.656400e+10,1.515300e+10,7.310000e+08,-1.028200e+10,2.081000e+09,-3.103000e+09,-1.064000e+09,1.370000e+09,-7.946000e+09,96798000000,42855000000,775000000.0,None,62957000000.0,6338000000.0,42855000000.0,7113000000.0,57005000000.0,48390000000.0,48390000000.0,-278000000.0,-3387000000.0,None,None,None,19916000000.0,96798000000.0,48408000000.0,39793000000.0,14567000000.0,None,43041000000.0,42855000000.0,7.751400e+10,5.790570e+11,3.560520e+11,6.338000e+09,1.466100e+10,9.414470e+11,2.054320e+11,3.482800e+10,1.656730e+11,1.867660e+11,125000000.0,-1.505300e+10,7.187900e+10,4.912600e+10,1.159290e+11,2.822000e+09,1.431300e+10,-1.505300e+10,6.298490e+11,1.515390e+11,1.054100e+10,2.784130e+11,3.013000e+09,2.853400e+10,2.033110e+11,3.603800e+10,2.667700e+10
2021Q3,1.370000e+08,2.621000e+09,2.057900e+10,-4.379900e+10,-6.821600e+10,7.596000e+09,3.114200e+10,1.029600e+10,2.833000e+09,5.510000e+10,6.962000e+09,1.599400e+10,-2.089500e+10,-3.052000e+09,-3.922000e+09,-3.522000e+09,1.802600e+10,-7.877000e+09,106020000000,31142000000,792000000.0,None,40977000000.0,6612000000.0,31142000000.0,7529000000.0,59552000000.0,48857000000.0,48857000000.0,1678000000.0,-2093000000.0,None,None,None,9753000000.0,106020000000.0,57163000000.0,46468000000.0,-7880000000.0,None,31224000000.0,31142000000.0,1.647600e+10,5.990000e+11,3.690770e+11,6.612000e+09,2.808100e+10,9.746890e+11,2.054320e+11,3.088600e+10,1.651100e+11,1.857510e+11,125000000.0,-1.465000e+09,7.739300e+10,5.939400e+10,1.386640e+11,4.852000e+09,1.858800e+10,-1.465000e+09,7.038680e+11,1.663340e+11,1.049300e+10,3.524760e+11,2.920000e+09,3.355700e+10,1.811250e+11,3.957700e+10,2.950200e+10
2021Q4,-7.590000e+08,1.208000e+09,3.108000e+09,-1.250100e+10,-5.573200e+10,-2.821800e+10,3.150400e+10,-9.440000e+08,2.880000e+08,5.139200e+10,1.590800e+10,2.752000e+09,-4.097600e+10,-8.870000e+08,-3.279000e+09,-2.255000e+09,4.182000e+09,-9.326000e+09,176587000000,31504000000,827000000.0,None,44485000000.0,2252000000.0,31504000000.0,7810000000.0,59047000000.0,49577000000.0,49577000000.0,431000000.0,-8356000000.0,None,None,None,12762000000.0,176587000000.0,127010000000.0,117540000000.0,-5092000000.0,None,31723000000.0,31504000000.0,1.675600e+10,5.833700e+11,3.873290e+11,2.252000e+09,2.000400e+10,9.729510e+11,2.054320e+11,2.267200e+10,1.642440e+11,1.712700e+11,123000000.0,1.765300e+10,7.974500e+10,5.841000e+10,1.349130e+11,3.371000e+09,2.031600e+10,1.765300e+10,6.994060e+11,1.682470e+11,8.674000e+09,3.704500e+11,N

In [199]:
import pandas_datareader as web
ciel = web.DataReader('CIEL3.SA', 'yahoo', start=f'01-01-{petr3_quarterly_balance_sheet.index[-1].year}').resample('Q')['Close'].quantile(0.8).pct_change().shift(-1)
ciel.index = pd.PeriodIndex(ciel.index, freq='Q')
ciel.name = '80th Closing Price Percentile'

In [200]:
dataset.join(ciel)

,Investments,Change To Liabilities,Total Cashflows From Investing Activities,Net Borrowings,Total Cash From Financing Activities,Change To Operating Activities,Net Income_x,Change In Cash,Effect Of Exchange Rate,Total Cash From Operating Activities,Depreciation,Other Cashflows From Investing Activities,Dividends Paid,Change To Inventory,Change To Account Receivables,Other Cashflows From Financing Activities,Change To Netincome,Capital Expenditures,Revenue,Earnings,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest_x,Net Income_y,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,Interest Expense,Extraordinary Items,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares,Intangible Assets,Total Liab,Total Stockholder Equity,Minority Interest_y,Other Current Liab,Total Assets,Common Stock,Other Current Assets,Retained Earnings,Other Liab,Good Will,Treasury Stock,Other Assets,Cash,Total Current Liabilities,Deferred Long Term Asset Charges,Short Long Term Debt,Other Stockholder Equity,Property Plant Equipment,Total Current Assets,Long Term Investments,Net Tangible Assets,Short Term Investments,Net Receivables,Long Term Debt,Inventory,Accounts Payable,80th Closing Price Percentile
2021Q2,3.540000e+08,-1.537000e+09,-5.187000e+09,-5.418100e+10,-6.552700e+10,-3.840000e+08,4.285500e+10,-1.902900e+10,-4.879000e+09,5.656400e+10,1.515300e+10,7.310000e+08,-1.028200e+10,2.081000e+09,-3.103000e+09,-1.064000e+09,1.370000e+09,-7.946000e+09,96798000000,42855000000,775000000.0,None,62957000000.0,6338000000.0,42855000000.0,7113000000.0,57005000000.0,48390000000.0,48390000000.0,-278000000.0,-3387000000.0,None,None,None,19916000000.0,96798000000.0,48408000000.0,39793000000.0,14567000000.0,None,43041000000.0,42855000000.0,7.751400e+10,5.790570e+11,3.560520e+11,6.338000e+09,1.466100e+10,9.414470e+11,2.054320e+11,3.482800e+10,1.656730e+11,1.867660e+11,125000000.0,-1.505300e+10,7.187900e+10,4.912600e+10,1.159290e+11,2.822000e+09,1.431300e+10,-1.505300e+10,6.298490e+11,1.515390e+11,1.054100e+10,2.784130e+11,3.013000e+09,2.853400e+10,2.033110e+11,3.603800e+10,2.667700e+10,-0.131500
2021Q3,1.370000e+08,2.621000e+09,2.057900e+10,-4.379900e+10,-6.821600e+10,7.596000e+09,3.114200e+10,1.029600e+10,2.833000e+09,5.510000e+10,6.962000e+09,1.599400e+10,-2.089500e+10,-3.052000e+09,-3.922000e+09,-3.522000e+09,1.802600e+10,-7.877000e+09,106020000000,31142000000,792000000.0,None,40977000000.0,6612000000.0,31142000000.0,7529000000.0,59552000000.0,48857000000.0,48857000000.0,1678000000.0,-2093000000.0,None,None,None,9753000000.0,106020000000.0,57163000000.0,46468000000.0,-7880000000.0,None,31224000000.0,31142000000.0,1.647600e+10,5.990000e+11,3.690770e+11,6.612000e+09,2.808100e+10,9.746890e+11,2.054320e+11,3.088600e+10,1.651100e+11,1.857510e+11,125000000.0,-1.465000e+09,7.739300e+10,5.939400e+10,1.386640e+11,4.852000e+09,1.858800e+10,-1.465000e+09,7.038680e+11,1.663340e+11,1.049300e+10,3.524760e+11,2.920000e+09,3.355700e+10,1.811250e+11,3.957700e+10,2.950200e+10,-0.314911
2021Q4,-7.590000e+08,1.208000e+09,3.108000e+09,-1.250100e+10,-5.573200e+10,-2.821800e+10,3.150400e+10,-9.440000e+08,2.880000e+08,5.139200e+10,1.590800e+10,2.752000e+09,-4.097600e+10,-8.870000e+08,-3.279000e+09,-2.255000e+09,4.182000e+09,-9.326000e+09,176587000000,31504000000,827000000.0,None,44485000000.0,2252000000.0,31504000000.0,7810000000.0,59047000000.0,49577000000.0,49577000000.0,431000000.0,-8356000000.0,None,None,None,12762000000.0,176587000000.0,127010000000.0,117540000000.0,-5092000000.0,None,31723000000.0,31504000000.0,1.675600e+10,5.833700e+11,3.873290e+11,2.252000e+09,2.000400e+10,9.729510e+11,2.054320e+11,2.267200e+10,1.642440e+11,1.712700e+11,123000000.0,1.765300e+10,7.974500e+10,5.841000e+10,1.349130e+11,3.371000e+09,2.031600e+10,1.765300e+10,6.9

In [222]:
# Este DF contém todos os tickers da BOVESPA
bovespa_companies = pd.read_csv('bovespa_companies.csv', usecols=[0],header=None, skiprows=[0,1],skipfooter=2)